### Cargo los datos

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_total = pd.read_csv("../data/train.csv")

### Desordeno los datos

In [2]:
df_total = df_total.sample(frac=1, random_state=100)

In [3]:
# Uso values para mandar todo a arrays de numpy
X = df_total[df_total.columns[1:]].values
y = df_total["label"].values.reshape(-1, 1)

X.shape, y.shape

((42000, 784), (42000, 1))

### Fijo un estado inicial para los parámetros

In [5]:
k = 1
alfa = 1

### Aplico PCA

In [7]:
import metnum

pca = metnum.PCA(1)
X = pca.transform(X)

### Aplico kNN

In [8]:
limit = int(0.8 * X.shape[0]) 

X_train, y_train = X[:limit], y[:limit]
X_val, y_val = X[limit:], y[limit:]

assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")

Ahora tengo 33600 instancias de entrenamiento y 8400 de validación


In [9]:
clf = metnum.KNNClassifier(k)
clf.fit(X_train, y_train)

In [10]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_val)

acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.22595238095238096
CPU times: user 34 s, sys: 427 ms, total: 34.5 s
Wall time: 39 s


### Pongo todo esto adentro de una función y sigo probando

In [13]:
def kNNconPCA(datos, k, alfa, semilla):
    df_train = pd.read_csv(datos)
    df_train = df_train.sample(frac=1, random_state=semilla)
    X = df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)
    pca = metnum.PCA(alfa)
    X = pca.transform(X)
    limit = int(0.8 * X.shape[0]) 
    X_train, y_train = X[:limit], y[:limit]
    X_val, y_val = X[limit:], y[limit:]
    clf = metnum.KNNClassifier(k)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    return acc

In [ ]:
k1_alfaVariable = []
alfa = [2,3,4,5,10,15,20,25,30]

for a in alfa:
    acc = kNNconPCA("../data/train.csv",1,a,100)
    k1_alfaVariable.append(acc)

In [19]:
k1_alfaVariable

[0.37285714285714283,
 0.4357142857142857,
 0.5620238095238095,
 0.6834523809523809,
 0.9158333333333334,
 0.9482142857142857,
 0.9644047619047619,
 0.9692857142857143,
 0.9726190476190476]

In [20]:
alfa = 35
kNNconPCA("../data/train.csv",1,alfa,100)

0.9728571428571429

In [21]:
alfa = 40
kNNconPCA("../data/train.csv",1,alfa,100)

0.9721428571428572

In [22]:
alfa = 34
kNNconPCA("../data/train.csv",1,alfa,100)

0.9730952380952381

In [23]:
alfa = 33
kNNconPCA("../data/train.csv",1,alfa,100)

0.973452380952381

In [24]:
alfa = 32
kNNconPCA("../data/train.csv",1,alfa,100)

0.9729761904761904

### Bueno. El mejor alfa es 33. Ahora fijo alfa y pruebo subiendo k.

In [26]:
from tqdm import tqdm

alfa = 33
ks = [2,3,4,5,6,7,8,9,10]
alfa33_kVariable = []

for k in tqdm(ks):
    acc = kNNconPCA("../data/train.csv",k,alfa,100)
    alfa33_kVariable.append(acc)

100%|██████████| 9/9 [14:39<00:00, 97.70s/it]


In [27]:
alfa33_kVariable

[0.9701190476190477,
 0.9760714285714286,
 0.9769047619047619,
 0.9760714285714286,
 0.9757142857142858,
 0.9744047619047619,
 0.9745238095238096,
 0.9729761904761904,
 0.9726190476190476]

### Listo. La mejor combinación es k = 4 y alfa = 33.